In [33]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import torch
import torchvision as tv

import noiseprofile as noise
import IntelSceneDataset as isds
import classifiers
from classifiers import SimpleClassifier

import denoise

In [19]:
noises = {"awn": noise.AdditiveWhiteNoise(0, 0.1),
          "srn": noise.ScalarRayleighNoise(1.0 / np.sqrt(2.0 * np.log(2.0))),
          "dropout": noise.DropoutNoise(0.05)
}

In [3]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
dbpath = "/mnt/hd-storage/IntelImageClassification"

cleanTr = isds.IntelSceneDataset(dbpath, transform=transform)

# dsTr = {"clean": cleanTr,
#         "awn": isds.IntelNoiseSceneDataset(cleanTr, noises["awn"]),
#         "srn": isds.IntelNoiseSceneDataset(cleanTr, noises["srn"]),
#         "dropout":  isds.IntelNoiseSceneDataset(cleanTr, noises["dropout"])}

In [5]:
clf = SimpleClassifier(cleanTr.categories)

In [66]:
dataloader = torch.utils.data.DataLoader(cleanTr,
                                           batch_size=4,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)
clf.train(dataloader, calcsnr=False)

In [67]:
clf.save("clean_sclf.pt")

In [68]:
C = clf.predict(dataloader, calcsnr=False)

In [69]:
print(C)

[[2271    0    0    0    0    0]
 [   2 2174    1    3   10    1]
 [   0    1 2269    2    1    1]
 [   1    1   11 2484    0   15]
 [   0    8    0    0 2374    0]
 [   1    0    6    5    0 2392]]


In [70]:
cleanTe = isds.IntelSceneDataset(dbpath, transform=transform, segment="test")
dsTe = {"clean": cleanTe,
        "awn": isds.IntelNoiseSceneDataset(cleanTe, noises["awn"]),
        "srn": isds.IntelNoiseSceneDataset(cleanTe, noises["srn"]),
        "dropout":  isds.IntelNoiseSceneDataset(cleanTe, noises["dropout"])}
C = {k: np.zeros((len(cleanTe.categories), len(cleanTe.categories)), np.uint) for k in dsTe.keys()}
snrFlag = {k: True for k in dsTe.keys()}
snrFlag["clean"] = False

In [71]:
for k, data in dsTe.items():
    dataloader = torch.utils.data.DataLoader(data,
                                           batch_size=32,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0)
    C[k] = clf.predict(dataloader, calcsnr=snrFlag[k], window=1)
    print("%s Accuracy: %.4f" % (k, np.sum(np.diag(C[k])) / np.sum(C[k])))

clean Accuracy: 0.7543


awn Accuracy: 0.6950


srn Accuracy: 0.4520


dropout Accuracy: 0.1557


In [72]:
import denoise
denoiser = denoise.CSC("test_dict.npy", dataloader, dtype=np.float64)

test_dict.npy
(10, 10, 3, 240)
float32


OutOfMemoryError: Out of memory allocating 691,200,000 bytes (allocated so far: 8,720,873,984 bytes).